# NFL Fantasy Football Quarterback Analysis
July 26, 2023

Alex Zaharia

# Introduction
I chose this topic because of the relevance football and fantasy football have in my life. I'm a huge Seahawks fan. I also play fantasy football every year with 9 of my friends. This year everyone is pooling in $100 each. The winner of the league gets the whole 1000 dollars and loser has to deal with a punishment (that we haven't come up with yet). It's very high stakes. This analysis helps me with analyzing the top quarterbacks for the 2023 season.

#Files/Sources:
### 1. weekly_data.csv (1999-2022, Kaggle)
- Weekly data of all active NFL offensive players from 1999-2022.
- https://www.kaggle.com/datasets/philiphyde1/nfl-stats-1999-2022?resource=download&select=weekly_data.xlsx

### 2. injury_data.csv (2023, ESPN)
- Injury data for players going into the 2023 season.
- https://www.espn.com/nfl/injuries

### 3. starters_data.csv (2022, CBS)
- Starting quarterbacks for each team going into the 2023 season.
- https://www.cbssports.com/nfl/news/2023-nfl-qb-power-rankings-lamar-jackson-aaron-rodgers-both-crack-top-10-in-first-rundown-of-all-32-starters/

## Structure:
**weekly_data.csv** has 56 columns, but we will only focus on 1 of them:
- name
- position
- fantasy_points
- Completions
- Passing Yards
- Carries
- Rushing Yards

**injury_data.csv** has 5 columns:
- name
- position
- date
- status
- comment

**starters_data.csv** has 2 columns:
- name
- team

## Considerations:
- **Excluded qbs drafted in 2023:** This analysis doesn't include any quarterbacks drafted in 2023. This is because newly drafted quarterbacks often get limited play time and won't start if there is a veteran quarterback already on their team.

- **Time Frame:** The analysis is based on data from 2022, the prior season. It's important to note that the NFL and player performance can evolve over time due to rule changes, coaching strategies, and player development. Therefore, historical data might not fully reflect the current trends in quarterback performance.

- **Trades & Transfers:** Player trades or transfers after the 2022 season are not accounted for. Its important to note that changes in a team can significantly influence a quarterback's fantasy points and overall performance. I decided not to include this information because it is very time consuming and I wanted prioritize other aspects of this analysis.


# Setup - Installation and Creating Data Frames

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u372-ga~us1-0ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameStatFunctions as statFuncs
from pyspark.sql.functions import col, abs
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, DoubleType
from google.colab import drive

drive.mount("/content/drive")
sc = SparkContext()
spark = SparkSession(sc)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Input the files

In [ ]:
schema = StructType([
    StructField("name", StringType(), False),
    StructField("position", StringType(), False),
    StructField("recent_team", StringType(), False),
    StructField("season", IntegerType(), False),
    StructField("week", IntegerType(), False),
    StructField("season_type", StringType(), False),
    StructField("completions", IntegerType(), False),
    StructField("attempts", IntegerType(), False),
    StructField("passing_yards", DoubleType(), False),
    StructField("passing_tds", IntegerType(), False),
    StructField("interceptions", IntegerType(), False),
    StructField("sacks", IntegerType(), False),
    StructField("sack_yards", IntegerType(), False),
    StructField("sack_fumbles", IntegerType(), False),
    StructField("sack_fumbles_lost", IntegerType(), False),
    StructField("passing_air_yards", DoubleType(), False),
    StructField("passing_yards_after_catch", DoubleType(), False),
    StructField("passing_first_downs", IntegerType(), False),
    StructField("passing_2pt_conversions", IntegerType(), False),
    StructField("carries", IntegerType(), False),
    StructField("rushing_yards", DoubleType(), False),
    StructField("rushing_tds", IntegerType(), False),
    StructField("rushing_fumbles", IntegerType(), False),
    StructField("rushing_fumbles_lost", IntegerType(), False),
    StructField("rushing_first_downs", IntegerType(), False),
    StructField("rushing_2pt_conversions", IntegerType(), False),
    StructField("receptions", IntegerType(), False),
    StructField("targets", IntegerType(), False),
    StructField("receiving_yards", DoubleType(), False),
    StructField("receiving_tds", IntegerType(), False),
    StructField("receiving_fumbles", IntegerType(), False),
    StructField("receiving_fumbles_lost", IntegerType(), False),
    StructField("receiving_air_yards", DoubleType(), False),
    StructField("receiving_yards_after_catch", DoubleType(), False),
    StructField("receiving_first_downs", IntegerType(), False),
    StructField("receiving_2pt_conversions", IntegerType(), False),
    StructField("target_share", DoubleType(), False),
    StructField("air_yards_share", DoubleType(), False),
    StructField("fantasy_points", DoubleType(), False),
    StructField("fantasy_points_ppr", DoubleType(), False),
    StructField("total_yards", DoubleType(), False),
    StructField("ypa", DoubleType(), False),
    StructField("ypc", DoubleType(), False),
    StructField("ypr", DoubleType(), False),
    StructField("touches", IntegerType(), False),
    StructField("count", IntegerType(), False),
    StructField("comp_percentage", DoubleType(), False),
    StructField("pass_td_percentage", DoubleType(), False),
    StructField("int_percentage", DoubleType(), False),
    StructField("rush_td_percentage", DoubleType(), False),
    StructField("rec_td_percentage", DoubleType(), False),
    StructField("total_tds", IntegerType(), False),
    StructField("td_percentage", DoubleType(), False),
    StructField("pr", DoubleType(), False),
    StructField("ht", StringType(), False),
    StructField("wt", StringType(), False)
])
weekly_df = spark.read.csv("/content/drive/MyDrive/NFLProject/weekly_data.csv", header=True, schema=schema)
weekly_df.select("*").show()

+----------+--------+-----------+------+----+-----------+-----------+--------+-------------+-----------+-------------+-----+----------+------------+-----------------+-----------------+-------------------------+-------------------+-----------------------+-------+-------------+-----------+---------------+--------------------+-------------------+-----------------------+----------+-------+---------------+-------------+-----------------+----------------------+-------------------+---------------------------+---------------------+-------------------------+-----------------+-----------------+--------------+------------------+-----------+---+----+-----+-------+-----+---------------+------------------+--------------+------------------+-----------------+---------+-------------+-----+---+---+
|      name|position|recent_team|season|week|season_type|completions|attempts|passing_yards|passing_tds|interceptions|sacks|sack_yards|sack_fumbles|sack_fumbles_lost|passing_air_yards|passing_yards_after_catc

In [ ]:
schema = StructType([
    StructField("name", StringType(), False),
    StructField("position", StringType(), False),
    StructField("date", StringType(), False),
    StructField("status", StringType(), False),
    StructField("comment", StringType(), False)
])
injury_df = spark.read.csv("/content/drive/MyDrive/NFLProject/injury_data.csv", header=True, schema=schema)
injury_df.select("*").show()

+-------------------+--------+------+---------------+--------------------+
|               name|position|  date|         status|             comment|
+-------------------+--------+------+---------------+--------------------+
|       Kyler Murray|      QB|Jul 12|   Questionable|Murray (knee) sai...|
|          Zach Ertz|      TE|Jun 12|   Questionable|"Coach Jonathan G...|
|         BJ Ojulari|      LB| Jun 1|   Questionable|Ojulari has not b...|
|   Garrett Williams|      CB|Apr 28|   Questionable|The Cardinals sel...|
|   Rashard Lawrence|      DT|Feb 13|   Questionable|                null|
|      Caleb Huntley|      RB|Jul 25|            Out|Huntley (Achilles...|
|    Calais Campbell|      DT|Jul 25|            Out|Campell (undisclo...|
|     Avery Williams|      CB|Jun 16|Injured Reserve|The Falcons place...|
|         Kyle Pitts|      TE|Jun 13|   Questionable|Pitts (knee) isn'...|
|        B.J. Baylor|      RB|May 16|Injured Reserve|Baylor (undisclos...|
|        Tyus Bowser|    

In [ ]:
schema = StructType([
    StructField("name", StringType(), False),
    StructField("team", StringType(), False),
    StructField("abbreviation", StringType(), False)
])
starters_df = spark.read.csv("/content/drive/MyDrive/NFLProject/starters_data.csv", header=True, schema=schema)
starters_df.select("*").show()

+----------------+--------------------+------------+
|            name|                team|abbreviation|
+----------------+--------------------+------------+
| Patrick Mahomes|  KANSAS CITY CHIEFS|          KC|
|      Joe Burrow|  CINCINNATI BENGALS|         CIN|
|     Jalen Hurts| PHILADELPHIA EAGLES|         PHI|
|      Josh Allen|       BUFFALO BILLS|         BUF|
|  Justin Herbert|LOS ANGELES CHARGERS|         LAC|
|   Aaron Rodgers|       NEW YORK JETS|         NYJ|
|   Lamar Jackson|    BALTIMORE RAVENS|         BAL|
| Trevor Lawrence|JACKSONVILLE JAGUARS|         JAX|
|    Dak Prescott|      DALLAS COWBOYS|         DAL|
|  Deshaun Watson|    CLEVELAND BROWNS|         CLE|
|    Kirk Cousins|   MINNESOTA VIKINGS|         MIN|
|     Brock Purdy| SAN FRANCISCO 49ERS|          SF|
|      Jared Goff|       DETROIT LIONS|         DET|
|      Geno Smith|    SEATTLE SEAHAWKS|         SEA|
|    Daniel Jones|     NEW YORK GIANTS|         NYG|
|   Justin Fields|       CHICAGO BEARS|       

# Primary Question:
**What are the top quarterback draft picks for the 2023 ESPN Fantasy Football season?**

##1. What are the names and total fantasy points for quarterbacks in 2022. Sort by fantasy points in descending order.

In [ ]:
qb_tfp_df = weekly_df.select("name","fantasy_points") \
            .filter((col("position") == "QB") & (col("season") == 2022)) \
            .groupBy("name").sum("fantasy_points") \
            .withColumnRenamed("sum(fantasy_points)", "totalFantasyPoints") \
            .sort(col("totalFantasyPoints").desc())

qb_tfp_df.show()

+---------------+------------------+
|           name|totalFantasyPoints|
+---------------+------------------+
|Patrick Mahomes|             416.4|
|     Josh Allen|395.52000000000004|
|    Jalen Hurts|378.03999999999996|
|     Joe Burrow|350.70000000000005|
|     Geno Smith|            303.88|
|  Justin Fields|            295.98|
|Trevor Lawrence|            295.62|
|   Kirk Cousins| 291.5799999999999|
|   Daniel Jones|             289.0|
|     Jared Goff|283.31999999999994|
| Justin Herbert|279.25999999999993|
|      Tom Brady|271.65999999999997|
|  Aaron Rodgers|             239.2|
|  Lamar Jackson|236.07999999999998|
| Tua Tagovailoa|230.92000000000002|
| Russell Wilson|224.75999999999993|
|     Derek Carr|219.08000000000004|
|   Kyler Murray|            200.52|
|   Dak Prescott|             198.6|
| Marcus Mariota|            196.56|
+---------------+------------------+
only showing top 20 rows



###Analysis
- This list provides us with the total fantasy points scored by all quarterbacks in the 2022 saeson. While it offers valuable insights into their overall performance, relying solely on these points might not present the full picture of their consistency and reliability.

- One crucial aspect to consider when using this data for drafting decisions is the potential impact of injuries or illnesses on a quarterback's performance. Throughout a season, quarterbacks may experience injuries or fall sick, leading to missed playing time on the field. The duration of these absences can vary, ranging from only a couple of days to the remainder of the season. Consequently, this can cause certain quarterbacks to have lower total fantasy points for a season, even if they may have performed exceptionally well in the games they did play.

- To resolve this issue and make a more informed decision on which quarterbacks to consider for drafting, we will find the names and average fantasy points for all quarterbacks sorted in descending order.



##2. What are the names and average fantasy points for all NFL quarterbacks sorted by points in descending order.

In [ ]:
qb_afp_df = weekly_df.select("name", "fantasy_points") \
                     .filter((col("position") == "QB") & (col("season") == 2022)) \
                     .groupBy("name").agg({"fantasy_points": "avg", "*": "count"}) \
                     .withColumnRenamed("avg(fantasy_points)", "avgFantasyPoints") \
                     .withColumnRenamed("count(1)", "gamesPlayed") \
                     .sort(col("avgFantasyPoints").desc())

qb_afp_df.show()

+---------------+------------------+-----------+
|           name|  avgFantasyPoints|gamesPlayed|
+---------------+------------------+-----------+
|    Jalen Hurts|25.202666666666666|         15|
|     Josh Allen|24.720000000000002|         16|
|Patrick Mahomes| 24.49411764705882|         17|
|     Joe Burrow|21.918750000000003|         16|
|     Davis Webb|             20.82|          1|
|  Justin Fields|19.732000000000003|         15|
|  Lamar Jackson|19.673333333333332|         12|
|     Sam Howell|             18.26|          1|
|   Kyler Murray| 18.22909090909091|         11|
|   Daniel Jones|           18.0625|         16|
|     Geno Smith| 17.87529411764706|         17|
| Tua Tagovailoa|17.763076923076923|         13|
|Trevor Lawrence|17.389411764705883|         17|
|   Kirk Cousins| 17.15176470588235|         17|
|     Jared Goff| 16.66588235294117|         17|
|   Dak Prescott|             16.55|         12|
| Justin Herbert|16.427058823529407|         17|
|      Tom Brady|15.

##Analysis

- Finding the average fantasy points across each game gives us a better representation of how quarterbacks perform when they are healthy and able to play.

- Having these two lists and comparing them to find quarterbacks on both lists shows us which quartbacks have the **highest** and **most consistent** fantasy productions.



##3. What are the names, total fantasy points, average fantasy points, and games played for all NFL quarterbacks in the total fantasy points and average fantasy points category sorted by average, then total fantasy points in descending order.

- Combine the last two tables.

In [ ]:
final_df = qb_tfp_df.join(qb_afp_df, qb_tfp_df["name"] == qb_afp_df["name"])
final_df = final_df.select(
    qb_tfp_df["name"],
    qb_tfp_df["totalFantasyPoints"],
    qb_afp_df["avgFantasyPoints"],
    qb_afp_df["gamesPlayed"]) \
    .sort(qb_tfp_df["totalFantasyPoints"].desc()) \
    .sort(qb_afp_df["avgFantasyPoints"].desc()) \
    .withColumnRenamed("name", "qbName") \
    .withColumnRenamed("count(1)", "gamesPlayed")
final_df.show(15)

+---------------+------------------+------------------+-----------+
|         qbName|totalFantasyPoints|  avgFantasyPoints|gamesPlayed|
+---------------+------------------+------------------+-----------+
|    Jalen Hurts|378.03999999999996|25.202666666666666|         15|
|     Josh Allen|395.52000000000004|24.720000000000002|         16|
|Patrick Mahomes|             416.4| 24.49411764705882|         17|
|     Joe Burrow|350.70000000000005|21.918750000000003|         16|
|     Davis Webb|             20.82|             20.82|          1|
|  Justin Fields|            295.98|19.732000000000003|         15|
|  Lamar Jackson|236.07999999999998|19.673333333333332|         12|
|     Sam Howell|             18.26|             18.26|          1|
|   Kyler Murray|            200.52| 18.22909090909091|         11|
|   Daniel Jones|             289.0|           18.0625|         16|
|     Geno Smith|            303.88| 17.87529411764706|         17|
| Tua Tagovailoa|230.92000000000002|17.763076923

###Analysis

- Notably, Davis Webb and Sam Howell stand out with notably high average points per game. However, to better understand the context behind their performance, it is essential to investigate why they achieved such high scores with only one game played each.

- The limited number of games played by these quarterbacks suggests that they might be either backups or starters who faced injuries during the season. As we aim to build our fantasy roster with players who receive substantial playtime and generate the highest overall fantasy production, it becomes crucial to determine if these quarterbacks are indeed regular starters or merely backups.


##4. What quarterbacks are starting in the 2023 season?
- Remove all players that aren't starters in 2023 from the table.


In [ ]:
final_df = final_df.join(starters_df, final_df["qbName"] == starters_df["name"])
final_df = final_df.select("qbName", "totalFantasyPoints", "avgFantasyPoints", "gamesPlayed") \
    .sort(qb_tfp_df["totalFantasyPoints"].desc()) \
    .sort(qb_afp_df["avgFantasyPoints"].desc())
final_df.show()

+---------------+------------------+------------------+-----------+
|         qbName|totalFantasyPoints|  avgFantasyPoints|gamesPlayed|
+---------------+------------------+------------------+-----------+
|    Jalen Hurts|378.03999999999996|25.202666666666666|         15|
|     Josh Allen|395.52000000000004|24.720000000000002|         16|
|Patrick Mahomes|             416.4| 24.49411764705882|         17|
|     Joe Burrow|350.70000000000005|21.918750000000003|         16|
|  Justin Fields|            295.98|19.732000000000003|         15|
|  Lamar Jackson|236.07999999999998|19.673333333333332|         12|
|     Sam Howell|             18.26|             18.26|          1|
|   Kyler Murray|            200.52| 18.22909090909091|         11|
|   Daniel Jones|             289.0|           18.0625|         16|
|     Geno Smith|            303.88| 17.87529411764706|         17|
| Tua Tagovailoa|230.92000000000002|17.763076923076923|         13|
|Trevor Lawrence|            295.62|17.389411764

###Analysis
- Davis Webb is the only player removed from the top 20 quarterbacks in this table. It is fair to assume that he was and still is (for 2023) a bench player who got the opportunity to start a single game. We don't want him in our roster.

- It is interesting though that Sam Howell is still on this list and holding a strong position within the top 20 quarterbacks. Considering he was a highly touted prospect coming out of college, his skill set and potential have likely translated well to the professional level.

- It's possible that he has shown significant growth and development since entering the league, proving himself as a reliable and competent starting quarterback.

##5. What injuries if any do these quarterbacks have coming into the 2023 season? Join with the previous table the date, status, and comments of these injuries for all quarterbacks.

In [ ]:
final_df = final_df.join(injury_df, final_df["qbName"] == injury_df["name"], "left_outer") \
                   .sort(qb_tfp_df["totalFantasyPoints"].desc()) \
                   .sort(qb_afp_df["avgFantasyPoints"].desc())
final_df = final_df.select("qbName", "totalFantasyPoints", "avgFantasyPoints", "gamesPlayed", "date", "status", "comment")
final_df.show(15)

+---------------+------------------+------------------+-----------+------+------------+--------------------+
|         qbName|totalFantasyPoints|  avgFantasyPoints|gamesPlayed|  date|      status|             comment|
+---------------+------------------+------------------+-----------+------+------------+--------------------+
|    Jalen Hurts|378.03999999999996|25.202666666666666|         15|  null|        null|                null|
|     Josh Allen|395.52000000000004|24.720000000000002|         16|  null|        null|                null|
|Patrick Mahomes|             416.4| 24.49411764705882|         17|  null|        null|                null|
|     Joe Burrow|350.70000000000005|21.918750000000003|         16|  null|        null|                null|
|  Justin Fields|            295.98|19.732000000000003|         15|  null|        null|                null|
|  Lamar Jackson|236.07999999999998|19.673333333333332|         12|  null|        null|                null|
|     Sam Howell|  

###Analysis
- Amongst the top 15 quarterbacks, Kyler Murray is the only one going into the 2023 season potentially injured (Questionable).

- Let's now retrieve the comment to see what is wrong with him and any other potentially injured quarterbacks.

##6. What are the names of all injured players and the comments on their injury reports.
- We want to know which quarterbacks are going into the 2023 season injured and which are not.

In [ ]:
temp_df = final_df
temp_df = temp_df.filter(final_df["comment"] != "null")
temp_df = temp_df.select("qbName", "comment")
for x in temp_df.collect():
  print(x[0], ":", x[1])

Kyler Murray : Murray (knee) said Wednesday that his goal is to be ready for Week 1, Tyler Drake of ArizonaSports.com reports.


###Analysis
- This comment made by Tyler Drake of ArizonaSports.com shows us that although Kyler Murray is facing a knee injury, he is likely to return for the start of the season.
- We should keep Kyler Murray in mind and stay up to date with any further news regarding his situation.
- It's not always ideal to draft a player who is coming into the season injured, as a reinjury is more likely to end their season or even career.
- We will, therefore, remove Kyler Murray and any other injured players from our roster.

##7. What starting quarterbacks are healthy?
- Remove all injured players from the table.
- We want a QB that is going into the season healthy and ready to play.

In [ ]:
temp_df = final_df.filter(col("comment").isNull())
final_df = temp_df.select("qbName", "totalFantasyPoints", "avgFantasyPoints", "gamesPlayed")
final_df.show()

+---------------+------------------+------------------+-----------+
|         qbName|totalFantasyPoints|  avgFantasyPoints|gamesPlayed|
+---------------+------------------+------------------+-----------+
|    Jalen Hurts|378.03999999999996|25.202666666666666|         15|
|     Josh Allen|395.52000000000004|24.720000000000002|         16|
|Patrick Mahomes|             416.4| 24.49411764705882|         17|
|     Joe Burrow|350.70000000000005|21.918750000000003|         16|
|  Justin Fields|            295.98|19.732000000000003|         15|
|  Lamar Jackson|236.07999999999998|19.673333333333332|         12|
|     Sam Howell|             18.26|             18.26|          1|
|   Daniel Jones|             289.0|           18.0625|         16|
|     Geno Smith|            303.88| 17.87529411764706|         17|
| Tua Tagovailoa|230.92000000000002|17.763076923076923|         13|
|Trevor Lawrence|            295.62|17.389411764705883|         17|
|   Kirk Cousins| 291.5799999999999| 17.15176470

##8. Find the average fantasy points each team's wide receivers produce.
- This is important because having good receivers means a quarterback will have higher chances of scoring more points in fantasy football.

In [ ]:
wr_afp_df = weekly_df.select("recent_team", "fantasy_points") \
                     .filter((col("position") == "WR") & (col("season") == 2022)) \
                     .groupBy("recent_team").agg({"fantasy_points": "avg"}) \
                     .withColumnRenamed("avg(fantasy_points)", "avgWRFantasyPoints") \
                     .sort("avgWRFantasyPoints", ascending = False)

wr_afp_df.show()

+-----------+------------------+
|recent_team|avgWRFantasyPoints|
+-----------+------------------+
|        CIN| 7.630967741935484|
|        MIN|              7.41|
|         LV| 7.242909090909091|
|        PHI| 7.156451612903225|
|        LAC| 6.812087912087911|
|        BUF|6.6085714285714285|
|        MIA| 6.309876543209876|
|        SEA| 6.012903225806452|
|        JAX| 5.928358208955224|
|        DET| 5.813235294117648|
|        DAL| 5.777049180327868|
|        WAS| 5.643055555555555|
|         SF| 5.473846153846154|
|         TB| 5.444303797468355|
|         GB| 5.297260273972602|
|        CLE|  5.19344262295082|
|         NO| 5.077272727272728|
|        ARI| 4.936231884057971|
|        IND| 4.931746031746032|
|        NYG|         4.8421875|
+-----------+------------------+
only showing top 20 rows



##9. What are the average fantasy points each team's running backs.
- This is important because when teams have good running backs, they will take away points from their quarterbacks.

In [ ]:
rb_afp_df = weekly_df.select("recent_team", "fantasy_points") \
                     .filter((col("position") == "RB") & (col("season") == 2022)) \
                     .groupBy("recent_team").agg({"fantasy_points": "avg"}) \
                     .withColumnRenamed("avg(fantasy_points)", "avgRBFantasyPoints") \
                     .sort("avgRBFantasyPoints", ascending = False)

rb_afp_df.show()

+-----------+------------------+
|recent_team|avgRBFantasyPoints|
+-----------+------------------+
|        DAL|              11.0|
|         SF| 9.014358974358974|
|        TEN| 8.960512820512822|
|         GB| 8.582051282051282|
|        DET| 8.528571428571427|
|        CLE| 8.144186046511628|
|        CIN| 7.878947368421054|
|         NE| 7.555263157894737|
|        MIN| 7.465789473684208|
|         KC| 7.322448979591837|
|        PIT|  7.25837837837838|
|        CAR| 7.108695652173913|
|        NYG| 7.092682926829268|
|        CHI|6.8769230769230765|
|        JAX| 6.765116279069767|
|        SEA| 6.764285714285713|
|         LV| 6.728571428571427|
|        MIA| 6.702499999999999|
|        LAC| 6.568518518518518|
|         TB| 6.515384615384615|
+-----------+------------------+
only showing top 20 rows



##10. Find the average fantasy points each team's tight ends produce.
- This is important because having good tight ends means a quarterback will have higher chances of scoring more points in fantasy football.

In [ ]:
te_afp_df = weekly_df.select("recent_team", "fantasy_points") \
                     .filter((col("position") == "TE") & (col("season") == 2022)) \
                     .groupBy("recent_team").agg({"fantasy_points": "avg"}) \
                     .withColumnRenamed("avg(fantasy_points)", "avgTEFantasyPoints") \
                     .sort("avgTEFantasyPoints", ascending = False)

te_afp_df.show()

+-----------+------------------+
|recent_team|avgTEFantasyPoints|
+-----------+------------------+
|         SF| 6.507999999999999|
|         KC| 6.202222222222223|
|         NO|5.6000000000000005|
|        BAL| 4.871428571428571|
|         LV| 4.695833333333334|
|        BUF| 4.581818181818182|
|        DET| 4.233333333333333|
|        ARI|3.9923076923076923|
|        PHI|3.9481481481481473|
|        DAL| 3.910810810810811|
|         LA|             3.776|
|        SEA| 3.753191489361703|
|        MIN| 3.718181818181818|
|        CLE|3.6121212121212123|
|        CHI|3.6034482758620694|
|        NYJ|          3.421875|
|        JAX| 3.394594594594595|
|        PIT|3.3027027027027023|
|        HOU| 3.189361702127659|
|        MIA| 3.144827586206896|
+-----------+------------------+
only showing top 20 rows



##11. Here we are joining the above three tables to have a side by side comparison.

In [ ]:
temp_df = final_df.join(starters_df, final_df["qbName"] == starters_df["name"])
temp_df = temp_df.select("qbName", "avgFantasyPoints", "gamesPlayed", "abbreviation")
temp_df = temp_df.join(wr_afp_df, temp_df["abbreviation"] == wr_afp_df["recent_team"])
temp_df = temp_df.join(rb_afp_df, temp_df["abbreviation"] == rb_afp_df["recent_team"])
temp_df = temp_df.join(te_afp_df, temp_df["abbreviation"] == te_afp_df["recent_team"])
temp_df = temp_df.select("qbName", "abbreviation", "gamesPlayed", "avgFantasyPoints", "avgWRFantasyPoints", "avgTEFantasyPoints", "avgRBFantasyPoints")
temp_df = temp_df.sort("avgFantasyPoints", ascending = False)

final_df = temp_df
final_df.show()

+---------------+------------+-----------+------------------+------------------+------------------+------------------+
|         qbName|abbreviation|gamesPlayed|  avgFantasyPoints|avgWRFantasyPoints|avgTEFantasyPoints|avgRBFantasyPoints|
+---------------+------------+-----------+------------------+------------------+------------------+------------------+
|    Jalen Hurts|         PHI|         15|25.202666666666666| 7.156451612903225|3.9481481481481473| 6.347916666666667|
|     Josh Allen|         BUF|         16|24.720000000000002|6.6085714285714285| 4.581818181818182| 5.485106382978723|
|Patrick Mahomes|          KC|         17| 24.49411764705882| 4.663291139240506| 6.202222222222223| 7.322448979591837|
|     Joe Burrow|         CIN|         16|21.918750000000003| 7.630967741935484| 2.725925925925926| 7.878947368421054|
|  Justin Fields|         CHI|         15|19.732000000000003| 3.158666666666667|3.6034482758620694|6.8769230769230765|
|  Lamar Jackson|         BAL|         12|19.673

##12. What is the overall fantasy points for these quarterbacks according to the formula below?

*Add the average fantasy points of qb to that of the qb's team's wide receivers and tight ends and subtract the points from running backs.*

- Having good WRs and TEs means a QB has the opportunity to score more points in a game.
- Having good RBs means a QB can't score as many points since the RB is running the ball more than the QB is passing it.


In [ ]:
temp_df = final_df
temp_df = temp_df.withColumn(
    "Overall Fantasy Points",
    temp_df["avgFantasyPoints"] +
    temp_df["avgWRFantasyPoints"] +
    temp_df["avgTEFantasyPoints"] -
    temp_df["avgRBFantasyPoints"]
)

temp_df = temp_df.orderBy(temp_df["Overall Fantasy Points"].desc())

qbs = temp_df.select("qbName", "Overall Fantasy Points").withColumnRenamed("qbName", "Name")

qbs.show()

+---------------+----------------------+
|           Name|Overall Fantasy Points|
+---------------+----------------------+
|     Josh Allen|     30.42528322741089|
|    Jalen Hurts|    29.959349761051374|
|Patrick Mahomes|    28.037182028929717|
|     Joe Burrow|    24.396696299440364|
|  Lamar Jackson|    23.196088164708854|
|     Geno Smith|    20.877103118529504|
|   Kirk Cousins|     20.81415705037996|
| Tua Tagovailoa|    20.515281052493698|
|Jimmy Garoppolo|    20.190170995670996|
|Trevor Lawrence|    19.947248289185936|
| Justin Herbert|    19.807771074241657|
|  Justin Fields|    19.617191865605662|
|     Sam Howell|    19.609873737373736|
|     Derek Carr|    19.466389844389845|
|   Daniel Jones|    18.708879573170734|
|     Jared Goff|     18.18387955182073|
| Russell Wilson|    16.424777962347726|
|  Aaron Rodgers|     15.58859050802139|
|   Dak Prescott|     15.23785999113868|
| Deshaun Watson|     14.92471112189374|
+---------------+----------------------+
only showing top

## 13. Break down the quarterbacks into 3 level tier system.

Categorize the quarterbacks into three tiers: "Elite," "Solid," and "Average." The tier cutoffs are determined dynamically based on the fantasy points of the quarterbacks. Cutoffs occur when there is a 3 point difference between quarterbacks or once we have deemed 3 QBs as "Elite" we will deem the next "Solid".

In [ ]:
from pyspark.sql.functions import col, when

# Initialize the cutoffs
temp = -1000000.0
tier1_cutoff = 0.0
tier2_cutoff = 0.0
count = 0

# Find the tier cutoffs
for x in qbs.collect():
  count += 1
  if temp == -1000000.0:
      temp = float(x[1])
  else:
      if (temp - float(x[1]) > 3.0):
          if tier1_cutoff == 0.0 or count == 3:
              tier1_cutoff = float(x[1])
          elif tier2_cutoff == 0.0:
              tier2_cutoff = float(x[1])
              break
          temp = float(x[1])

# Use DataFrame transformations to create a new column for the tier
qbs_df = qbs.withColumn("Pick Quality",
                        when(col("Overall Fantasy Points") > tier1_cutoff, "Elite")
                        .when(col("Overall Fantasy Points") > tier2_cutoff, "Solid")
                        .otherwise("Average"))

# Show the DataFrame
qbs_df.show(10)

+---------------+----------------------+------------+
|           Name|Overall Fantasy Points|Pick Quality|
+---------------+----------------------+------------+
|     Josh Allen|     30.42528322741089|       Elite|
|    Jalen Hurts|    29.959349761051374|       Elite|
|Patrick Mahomes|    28.037182028929717|       Elite|
|     Joe Burrow|    24.396696299440364|       Solid|
|  Lamar Jackson|    23.196088164708854|       Solid|
|     Geno Smith|    20.877103118529504|     Average|
|   Kirk Cousins|     20.81415705037996|     Average|
| Tua Tagovailoa|    20.515281052493698|     Average|
|Jimmy Garoppolo|    20.190170995670996|     Average|
|Trevor Lawrence|    19.947248289185936|     Average|
+---------------+----------------------+------------+
only showing top 10 rows



##14. Dynamically classify quarterbacks based on performance ratios.
1. The quarterbacks are categorized into different types such as "Elite Dual Threat," "Dual Threat," "Elite Rusher," "Rusher," "Elite Pocket Passer," "Pocket Passer," "Scrambler," or "None," depending on their average completions, passing yards, carries, and rushing yards ratios relative to the overall average for all players.

2. In the formula for determining the category each quarterback falls in I am calculating the ratio between each individual quarterback's average key statistics to those of the average of of all quarterbacks' average key statistic.


> - If avgCompletionsRatio is greater than or equal to 1.2 and avgPassingYardsRatio is greater than or equal to 1.2 and avgCarriesRatio is greater than or equal to 1.2 and avgRushingYardsRatio is greater than or equal to 1.2:

> > > The Quarterback is categorized as "Elite Dual Threat"

> - Else, if avgCompletionsRatio is greater than or equal to 1.0 and avgPassingYardsRatio is greater than or equal to 1.0 and avgCarriesRatio is greater than or equal to 1.0 and avgRushingYardsRatio is greater than or equal to 1.0:

> > > The Quarterback is categorized as "Dual Threat"

> - Else, if avgCarriesRatio is greater than or equal to 1.2 and avgRushingYardsRatio is greater than or equal to 1.2:

> > > The Quarterback is categorized as "Elite Rusher"

> - Else, if avgCarriesRatio is greater than or equal to 1.0 and avgRushingYardsRatio is greater than or equal to 1.0:

> > > The Quarterback is categorized as "Rusher"

> - Else, if avgCompletionsRatio is greater than or equal to 1.2 and avgPassingYardsRatio is greater than or equal to 1.2:

> > > The Quarterback is categorized as "Elite Pocket Passer"

> - Else, if avgCompletionsRatio is greater than or equal to 1.0 and avgPassingYardsRatio is greater than or equal to 1.0:

> > > The Quarterback is categorized as "Pocket Passer"

> - Else, if avgCarriesRatio is greater than or equal to 1.0:

> > > The Quarterback is categorized as "Scrambler"

> - Else:

> > > The Quarterback is categorized as "None"






In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf

# Function to categorize quarterbacks based on their average performance ratios
def categorize_quarterback(avgCompletionsRatio, avgPassingYardsRatio, avgCarriesRatio, avgRushingYardsRatio):
    if avgCompletionsRatio >= 1.2 and avgPassingYardsRatio >= 1.2 and avgCarriesRatio >= 1.2 and avgRushingYardsRatio >= 1.2:
        return "Elite Dual Threat"
    elif avgCompletionsRatio >= 1.0 and avgPassingYardsRatio >= 1.0 and avgCarriesRatio >= 1.0 and avgRushingYardsRatio >= 1.0:
        return "Dual Threat"
    elif avgCarriesRatio >= 1.2 and avgRushingYardsRatio >= 1.2:
        return "Elite Rusher"
    elif avgCarriesRatio >= 1.0 and avgRushingYardsRatio >= 1.0:
        return "Rusher"
    elif avgCompletionsRatio >= 1.2 and avgPassingYardsRatio >= 1.2:
        return "Elite Pocket Passer"
    elif avgCompletionsRatio >= 1.0 and avgPassingYardsRatio >= 1.0:
        return "Pocket Passer"
    elif avgCarriesRatio >= 1.0:
        return "Scrambler"
    else:
        return "None"

# Filter and aggregate quarterback abilities
abilities_df = weekly_df.filter((col("position") == "QB") & (col("season") == 2022)) \
                     .groupBy("name").agg({"completions": "avg", "passing_yards": "avg", "carries": "avg", "rushing_yards": "avg"}) \
                     .withColumnRenamed("avg(carries)", "avgCarries") \
                     .withColumnRenamed("avg(rushing_yards)", "avgRushingYards") \
                     .withColumnRenamed("avg(completions)", "avgCompletions") \
                     .withColumnRenamed("avg(passing_yards)", "avgPassingYards")

# Join the abilities DataFrame with the quarterbacks DataFrame
temp_df = qbs_df.join(abilities_df, qbs_df["Name"] == abilities_df["name"])

# Sort the DataFrame by Overall Fantasy Points in descending order and rename the "category" column to "Type"
temp_df = temp_df.sort("Overall Fantasy Points", ascending=False).withColumnRenamed("category", "Type")

# Select relevant columns for the final output
temp_df = temp_df.select(qbs_df["Name"], "Overall Fantasy Points", "Pick Quality","avgCarries","avgRushingYards", "avgCompletions", "avgPassingYards")

# Calculating averages across all players
totalCompletions = 0
totalPassingYards = 0
totalCarries = 0
totalRushingYards = 0
count = 0

# Calculate the totals and counts
for x in temp_df.collect():
    totalCompletions += x[5]
    totalPassingYards += x[6]
    totalCarries += x[3]
    totalRushingYards += x[4]
    count += 1
averageCompletions = totalCompletions / count
averagePassingYards = totalPassingYards / count
averageCarries = totalCarries / count
averageRushingYards = totalRushingYards / count

# Calculate the ratios for each Type relative to the average values
temp_df = temp_df.withColumn("avgCarriesRatio", col("avgCarries") / averageCarries) \
                 .withColumn("avgPassingYardsRatio", col("avgPassingYards") / averagePassingYards) \
                 .withColumn("avgCompletionsRatio", col("avgCompletions") / averageCompletions) \
                 .withColumn("avgRushingYardsRatio", col("avgRushingYards") / averageRushingYards)

# Select relevant columns for the final output
temp_df.select("Name", "Overall Fantasy Points", "Pick Quality", "avgCompletionsRatio", "avgPassingYardsRatio", "avgCarriesRatio", "avgRushingYardsRatio")

# Define a User-Defined Function (udf) to categorize quarterbacks based on the ratios
categorize_udf = udf(categorize_quarterback, StringType())

# Apply the UDF to create the "Type" column based on the calculated ratios
temp_df = temp_df.withColumn("Type", categorize_udf(col("avgCompletionsRatio"), col("avgPassingYardsRatio"), col("avgCarriesRatio"), col("avgRushingYardsRatio")))

# Select the relevant columns for the final output and show the results
final_df = temp_df.select("Name", "Overall Fantasy Points", "Pick Quality", "Type")
final_df.show(10)

+---------------+----------------------+------------+-------------------+
|           Name|Overall Fantasy Points|Pick Quality|               Type|
+---------------+----------------------+------------+-------------------+
|     Josh Allen|     30.42528322741089|       Elite|        Dual Threat|
|    Jalen Hurts|    29.959349761051374|       Elite|        Dual Threat|
|Patrick Mahomes|    28.037182028929717|       Elite|Elite Pocket Passer|
|     Joe Burrow|    24.396696299440364|       Solid|Elite Pocket Passer|
|  Lamar Jackson|    23.196088164708854|       Solid|       Elite Rusher|
|     Geno Smith|    20.877103118529504|     Average|      Pocket Passer|
|   Kirk Cousins|     20.81415705037996|     Average|Elite Pocket Passer|
| Tua Tagovailoa|    20.515281052493698|     Average|      Pocket Passer|
|Jimmy Garoppolo|    20.190170995670996|     Average|               None|
|Trevor Lawrence|    19.947248289185936|     Average|      Pocket Passer|
+---------------+---------------------

## 15. Add Descriptions for QB Types (Dynamic classification system for QBs).
The QB Type, QB Type Description table shows what each category of QB Type represents.

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf

descriptions = {
    "Elite Dual Threat": "Can pass and run effectively, an elite dual threat quarterback.",
    "Dual Threat": "Capable of both passing and running the ball effectively.",
    "Elite Rusher": "An elite rushing quarterback, excelling in running plays.",
    "Rusher": "A quarterback who performs well in running plays.",
    "Elite Pocket Passer": "An elite quarterback with exceptional passing abilities from the pocket.",
    "Pocket Passer": "A quarterback who relies on passing skills from the pocket.",
    "Scrambler": "Known for scrambling and running the ball when needed.",
    "None": "Does not fit into any specific category based on the average ratios."
}

# Define a function to map the type to its description
def map_ability_to_description(type):
    return descriptions.get(type, "None")

# Select the relevant columns for the final output and show the results
final_df = final_df.select("Name", "Overall Fantasy Points", "Pick Quality", "Type") \
       .withColumn("QB Type Description", F.udf(map_ability_to_description, StringType())("Type")) \
       .withColumnRenamed("Type", "QB Type")
final_df.show(10)

# Print the QB Types and QB Type Descriptions
max_qb_type_width = max(len(x) for x in descriptions.keys())
max_description_width = max(len(y) for y in descriptions.values())

output_text = f"+{'-' * (max_qb_type_width + 2)}+{'-' * (max_description_width + 2)}+\n"
output_text += f"| {'QB Type':<{max_qb_type_width}} | {'QB Type Description':<{max_description_width}} |\n"
output_text += f"+{'-' * (max_qb_type_width + 2)}+{'-' * (max_description_width + 2)}+\n"

for x, y in descriptions.items():
    output_text += f"| {x:<{max_qb_type_width}} | {y:<{max_description_width}} |\n"

output_text += f"+{'-' * (max_qb_type_width + 2)}+{'-' * (max_description_width + 2)}+\n"
print(output_text)

+---------------+----------------------+------------+-------------------+--------------------+
|           Name|Overall Fantasy Points|Pick Quality|            QB Type| QB Type Description|
+---------------+----------------------+------------+-------------------+--------------------+
|     Josh Allen|     30.42528322741089|       Elite|        Dual Threat|Capable of both p...|
|    Jalen Hurts|    29.959349761051374|       Elite|        Dual Threat|Capable of both p...|
|Patrick Mahomes|    28.037182028929717|       Elite|Elite Pocket Passer|An elite quarterb...|
|     Joe Burrow|    24.396696299440364|       Solid|Elite Pocket Passer|An elite quarterb...|
|  Lamar Jackson|    23.196088164708854|       Solid|       Elite Rusher|An elite rushing ...|
|     Geno Smith|    20.877103118529504|     Average|      Pocket Passer|A quarterback who...|
|   Kirk Cousins|     20.81415705037996|     Average|Elite Pocket Passer|An elite quarterb...|
| Tua Tagovailoa|    20.515281052493698|     Avera

#Conclusion:

In analyzing the fantasy football performance of quarterbacks, it's essential to acknowledge that real-life football is a multifaceted team sport where a quarterback's performance is influenced by various factors beyond the quality of their teammates. Furthermore, individual player performance can be subject to numerous variables such as opposing team defenses, weather conditions, injuries, and coaching strategies.

While this analysis has provided valuable insights into the categorization of quarterbacks based on their abilities and overall fantasy performance, there are several considerations for future analysis to further enhance the understanding of quarterback performance in fantasy football:

- **Schedules and Opposing Defenses**: To gain deeper insights into quarterback performance, it would be beneficial to incorporate the quality of opposing defenses that a team faces. This could involve developing a ranking system or utilizing key statistics to measure the defensive capabilities of all NFL teams. By joining this information with the quarterback data, a more comprehensive understanding of how varying defensive strengths impact quarterback performance can be achieved.

- **Injury History**: To account for the impact of injuries on quarterback performance, obtaining more detailed injury data could be crucial. This would involve considering specific injury types, recovery times, and other important details related to injuries. Understanding how injuries affect a quarterback's consistency and long-term performance could provide valuable insights for fantasy football strategies.

- **Weather Conditions**: Weather conditions play a significant role in NFL games, affecting quarterbacks' passing accuracy and overall performance. Incorporating data on weather patterns at every active NFL stadium on a week-by-week basis would enable the examination of how different weather conditions impact quarterback fantasy output. This consideration could lead to more informed decisions when selecting quarterbacks for specific game scenarios.

- **Social Media Analysis**: Monitoring social media activity of quarterbacks could offer insights into their mental state, public perception, and possible distractions that may influence their performance. Analyzing social media data could provide a window into the quarterback's off-field activities and their dedication to improving their game. This aspect of analysis could help assess the psychological and emotional factors that might affect their on-field performance.

In conclusion, while this analysis has categorized quarterbacks based on their abilities and fantasy football performance, it is vital to consider the broader context of real-life football and the numerous factors that influence quarterback play. Incorporating additional data and exploring different dimensions could lead to a more comprehensive and nuanced understanding of quarterback performance in the realm of fantasy football. These potential considerations open up exciting avenues for further research and could empower fantasy football enthusiasts to make more informed decisions when drafting quarterbacks for their teams.